In [47]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'label_featmat'

In [ ]:
from Bio import SeqIO
file = '../xlink/data/fastas/tett_marcotte_annotations.fasta'


In [49]:
!ls

build_fmat.ipynb	       join_feats.ipynb
clean_pipeline.sh	       label_fmat.ipynb
cluster.ipynb		       lab_notebook.sh
cmplx_notes.sh		       leca_circpleplot.Rmd
cross_validation.ipynb	       LECA.Rmd
debug.ipynb		       localization_labels.html
eval_group_merge.ipynb	       localization_labels.nb.html
eval_multigroup_ppi.ipynb      localization_labels.pdf
feature_selection.ipynb        localization_labels.Rmd
filter_chloroplasts.ipynb      make_cmplx_files.ipynb
filter_eluts.ipynb	       make_pkl.ipynb
filter_leca_nogs.ipynb	       map_entrez_to_eggnog.ipynb
fix_tryb2.ipynb		       map_og_peps.ipynb
format_annotations.ipynb       og_coverage_plots.Rmd
format_old_apms_featmat.ipynb  plot_heatmap.ipynb
gather_pep_data.ipynb	       ppi_network_plots.Rmd
gather_prot_lists.ipynb        predict_ppis.ipynb
get_apms_scores.ipynb	       reorder_fracs.ipynb
get_nog_coverage.ipynb	       sandbox.ipynb
get_nog_family_sizes.ipynb     sanity_checks.ipynb
get_presence_absence.ipynb     template.R

In [2]:
df = pd.DataFrame(np.random.randn(100, 4), columns=list('ABCD'))
df

,A,B,C,D
0,1.480560,-0.221869,0.518213,1.025696
1,-0.472902,-0.393165,0.809715,-0.302157
2,-0.717116,-0.045341,-0.419402,-0.966210
3,0.252466,0.407023,1.566091,-0.800921
4,0.061788,-0.091372,0.723425,0.318874
...,...,...,...,...
95,-0.783479,0.141852,0.399782,-2.300502
96,-0.531800,-0.812779,-0.882211,-0.012783
97,-1.693391,-0.407102,-0.594677,-1.238593
98,0.163797,-0.781257,0.195418,-0.414813


In [38]:
grp1 = ['Group 1']*30
grp2 = ['Group 2']*50
grp3 = ['Group 3']*20
grps = grp1+grp2+grp3
len(grps)

100

In [39]:
df['group'] = grps

In [40]:
score = 'A'
gb = df.groupby(['group'])
counts = gb.size().to_frame(name='counts')
agg_res = (counts
               #.join(gb.agg({'fold': lambda x: ', '.join(set(x.astype(str).dropna()))}))
               .join(gb.agg({f'{score}': 'mean'}).rename(columns={f'{score}': f'mean_{score}'}))
               .join(gb.agg({f'{score}': 'min'}).rename(columns={f'{score}': f'min_{score}'}))
               .join(gb.agg({f'{score}': 'max'}).rename(columns={f'{score}': f'max_{score}'}))
               .join(gb.agg({f'{score}': 'std'}).rename(columns={f'{score}': f'stdev'}))
               #.join(gb.agg({f'{score}': 'var'}).rename(columns={f'{score}': f'rsd'}))
               #.join(gb.agg({'rsd': lambda x: np.std(x) / np.mean(x)}))
               .sort_values(['counts', 'stdev', f'mean_{score}'], key=abs, ascending=[False, True, False])
               .reset_index()
              )


agg_res.sort_values(['counts', 'rsd', f'mean_{score}'], key=abs, ascending=[False, True, False])
agg_res

,group,counts,mean_A,min_A,max_A,stdev,rsd
0,Group 2,50,0.070159,-2.557286,1.726277,1.005781,14.335756
1,Group 1,30,-0.526927,-2.188379,1.480560,0.967570,-1.836251
2,Group 3,20,0.247797,-1.693391,2.237494,0.946351,3.819061


In [44]:
final = (agg_res
         .assign(rsd_sort=agg_res['rsd'].abs(), score_sort=agg_res[f'mean_{score}'].abs())
         .sort_values(['counts', 'rsd_sort', 'score_sort'],ascending=[False, True, False])
         .drop(['rsd_sort', 'score_sort'], 1)
         .reset_index()
        )
final

/tmp/ipykernel_3096683/1314021587.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .drop(['rsd_sort', 'score_sort'], 1)


,group,counts,mean_A,min_A,max_A,stdev,rsd
1,Group 1,30,-0.526927,-2.188379,1.480560,0.967570,-1.836251
2,Group 3,20,0.247797,-1.693391,2.237494,0.946351,3.819061
0,Group 2,50,0.070159,-2.557286,1.726277,1.005781,14.335756


In [43]:
df = (df.assign(A=df['dist'].abs())
        .sort_values(['A','taking'],ascending=[True, False])
        .drop('A', 1))

KeyError: ('counts', 'rsd')

In [20]:
agg_res

,group,counts,mean_A,min_A,max_A,stdev,rsd
0,Group 2,50,0.167313,-1.830466,2.237494,0.995874,5.952167
1,Group 1,50,-0.314191,-2.557286,1.480560,0.995968,-3.169940


In [47]:
data = pd.read_csv('../ppi_ml/results/walktrap_ExtraTreesClassifier_100feats.csv')

In [50]:
string = 'cilia'
sub_df = data[data.apply(lambda r: r.str.contains(string, case=False).any(), axis=1)]
sub_df.to_csv(f'../ppi_ml/results/walktrap_ExtraTreesClassifier_100feats_{string}.csv', index=False)

In [25]:
import re
import pandas as pd

def format_results(ogs):
    # format eggnog results
    res_fmt = []
    results = ogs.split(',')
    for i in results:
        og = re.search(".*(?=@)", i)[0]
        if og and og[0].isdigit():
            og = 'ENOG50'+og
        lvl = re.search("(?<=@).*(?=\|)", i)[0]
        taxa = re.search("(?<=\|).*", i)[0]
        res_fmt.append([og, lvl, taxa])
    return(res_fmt)

In [34]:
infile = '../../misc/test/mouse.hmmer.emapper.annotations'
format_uniprot_id = False
level = 1

In [35]:
enog = pd.read_csv(infile, sep='\t')
enog

,#query_name,seed_eggNOG_ortholog,seed_ortholog_evalue,seed_ortholog_score,eggNOG OGs,narr_og_name,narr_og_cat,narr_og_desc,best_og_name,best_og_cat,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
0,tr|A0A075B5I2|A0A075B5I2_MOUSE,9555.ENSPANP00000007068,7.200000e-58,182.4,"2E2YC@1|root,2SA48@2759|Eukaryota,3A62F@33154|...",36CIA@314294|Cercopithecoidea,S,T cell receptor beta variable 3-1,3BTC3@33208|Metazoa,S,...,-,-,-,-,-,-,-,-,-,V-set
1,tr|A0A075B5I3|A0A075B5I3_MOUSE,10042.XP_006997171.1,1.500000e-119,381.3,"2E2YC@1|root,2SA48@2759|Eukaryota,3A9AN@33154|...",35TZ5@314146|Euarchontoglires,T,Immunoglobulin V-Type,3BUCM@33208|Metazoa,S,...,-,-,-,-,-,-,-,-,-,V-set
2,tr|A0A075B5I8|A0A075B5I8_MOUSE,28737.XP_006887304.1,5.800000e-132,421.5,"2E2YC@1|root,2SA48@2759|Eukaryota,3AMTR@33154|...",357IY@311790|Afrotheria,S,Immunoglobulin V-set domain,3C0IF@33208|Metazoa,S,...,-,-,-,-,-,-,-,-,-,V-set
3,tr|A0A075B5I9|A0A075B5I9_MOUSE,28737.XP_006887304.1,5.200000e-118,378.3,"2E2YC@1|root,2SA48@2759|Eukaryota,3AMTR@33154|...",357IY@311790|Afrotheria,S,Immunoglobulin V-set domain,3C0IF@33208|Metazoa,S,...,-,-,-,-,-,-,-,-,-,V-set
4,tr|A0A075B5J0|A0A075B5J0_MOUSE,28737.XP_006887304.1,2.800000e-124,398.3,"2E2YC@1|root,2SA48@2759|Eukaryota,3AMTR@33154|...",357IY@311790|Afrotheria,S,Immunoglobulin V-set domain,3C0IF@33208|Metazoa,S,...,-,-,-,-,-,-,-,-,-,V-set
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21372,tr|W4VSP5|W4VSP5_MOUSE,10090.ENSMUSP00000128012,5.000000e-205,671.3,"29630@1|root,2R0MZ@2759|Eukaryota,3AG0E@33154|...",35TXF@314146|Euarchontoglires,I,"Vomeronasal organ pheromone receptor family, V1R",3BWW9@33208|Metazoa,T,...,ko:K04614,-,-,-,-,"ko00000,ko04030",9.A.14.12,-,-,"7tm_1,TAS2R,V1R"
21373,tr|W4VSP6|W4VSP6_MOUSE,10090.ENSMUSP00000128877,2.600000e-281,925.1,"COG0657@1|root,KOG1515@2759|Eukaryota,3AF66@33...",4Q7F4@9989|Rodentia,V,alpha/beta hydrolase fold,3BX40@33208|Metazoa,V,...,ko:K14350,-,-,-,-,"ko00000,ko01000",-,-,-,"Abhydrolase_3,COesterase,DUF2424,Say1_Mug180"
21374,tr|W4VSP7|W4VSP7_MOUSE,10029.XP_007639544.1,7.900000e-89,286.7,"KOG4726@1|root,KOG4726@2759|Eukaryota,39DSA@33...",4Q8RT@9989|Rodentia,W,"Keratin, high sulfur B2 protein",3BMXG@33208|Metazoa,W,...,-,-,-,-,-,-,-,-,-,"Keratin_B2_2,PMG"
21375,tr|W4VSP8|W4VSP8_MOUSE,10090.ENSMUSP00000134901,3.600000e-206,676.9,"29630@1|root,2SNRJ@2759|Eukaryota,3AFSD@33154|...",35EQJ@314146|Euarchontoglires,T,vomeronasal,3BX57@33208|Metazoa,T,...,ko:K04614,-,-,-,-,"ko00000,ko04030",9.A.14.12,-,-,"7tm_1,TAS2R,V1R"


In [36]:
enog = pd.read_csv(infile, sep='\t')
enog.columns.values[0] = 'ProteinID'
enog.columns.values[4] = 'ID'
enog = enog[['ProteinID','ID']]

results_dict = dict()
for i in range(len(enog)):
    prot_id = enog['ProteinID'][i]
    if format_uniprot_id:
        prot_id = re.search("(?<=\|).*(?=\|)", prot_id)[0]
    ogs = enog['ID'][i]
    res_fmt = format_results(ogs)
    results_dict[prot_id] = res_fmt

df = pd.DataFrame.from_dict(results_dict, orient='index')
df = (df
      .reset_index()
      .melt(id_vars='index')
      .drop(labels='variable',axis=1)
      .dropna()
      .rename(columns={'index':'ProteinID'})
     )

df[['ID','level','taxonomy']] = pd.DataFrame(df.value.tolist(), index=df.index)
df['level'] = df['level'].astype(int)
df.drop(labels='value', axis=1, inplace=True)

level_df = df[df.level==level]
out_df = level_df[['ProteinID', 'ID']]

In [37]:
df

,ProteinID,ID,level,taxonomy
0,tr|A0A075B5I2|A0A075B5I2_MOUSE,ENOG502E2YC,1,root
1,tr|A0A075B5I3|A0A075B5I3_MOUSE,ENOG502E2YC,1,root
2,tr|A0A075B5I8|A0A075B5I8_MOUSE,ENOG502E2YC,1,root
3,tr|A0A075B5I9|A0A075B5I9_MOUSE,ENOG502E2YC,1,root
4,tr|A0A075B5J0|A0A075B5J0_MOUSE,ENOG502E2YC,1,root
...,...,...,...,...
334936,sp|Q8CG65|SSPO_MOUSE,ENOG503J58U,40674,Mammalia
353901,sp|Q80T14|FRAS1_MOUSE,ENOG5035MF4,314146,Euarchontoglires
356313,sp|Q8CG65|SSPO_MOUSE,ENOG5035HAP,314146,Euarchontoglires
375278,sp|Q80T14|FRAS1_MOUSE,ENOG504PW3P,9989,Rodentia


In [143]:
level_df[level_df.level==1]['taxonomy'].unique()[0]

'root'

In [3]:
import pandas as pd
from functools import reduce
infile = '../ppi_ml/results/walktrap/walktrap_ExtraTreesClassifier_100feats_fdr10.csv'
gold_std_file = '../ppi_ml/data/gold_stds/all.gold.cmplx.noRibos.merged.txt'

In [4]:
res = pd.read_csv(infile)
cuts = [c for c in res.columns.values.tolist() if 'cut' in c]
df_walk = res[['ID'] + cuts]
df_walk

,ID,cut_154,cut_298,cut_442,cut_817,cut_1193,cut_1568,cut_1944,cut_2320,cut_2695,cut_3071
0,KOG0177,0,0,0,0,0,0,0,0,0,0
1,KOG0182,0,0,0,0,0,0,0,0,0,0
2,KOG0184,0,0,0,0,0,0,0,0,0,0
3,KOG0176,0,0,0,0,0,0,0,0,0,13
4,KOG0179,0,0,0,0,0,0,0,0,0,13
...,...,...,...,...,...,...,...,...,...,...,...
3442,KOG2495,120,293,425,800,1175,1550,1926,2302,2677,3053
3443,KOG3625,1,294,426,801,1177,1552,1928,2304,2679,3055
3444,ENOG502QRMS,1,295,432,807,1183,1558,1934,2310,2685,3061
3445,KOG4475,1,296,435,810,1186,1561,1937,2313,2688,3064


In [22]:
df_complx_sizes = df_walk.groupby(cuts).size()
df_complx_sizes

cut_154  cut_298  cut_442  cut_817  cut_1193  cut_1568  cut_1944  cut_2320  cut_2695  cut_3071
0        0        0        0        0         0         0         0         0         0           3
                                                                                      13          2
                                                                                      21          5
                                                                  1         1         1           3
                                                                                      776         1
                                                                                                 ..
149      277      405      779      1154      1529      1905      2281      2656      3032        1
150      278      406      780      1155      1530      1906      2282      2657      3033        1
151      286      418      792      1167      1542      1918      2294      2669      3045        1
152  

In [5]:
with open(gold_std_file, 'r') as f:
    ppis = f.read().splitlines()
    prot_list = []
    cmplx_list = []
    for i, p in enumerate(ppis):
        ogs = p.split(' ')
        for prot in ogs:
            prot_list.append(prot)
            cmplx_list.append(i+1)

df_gold = pd.DataFrame()
df_gold['ID'] = prot_list
df_gold['cmplx'] = cmplx_list
df_gold = df_gold[df_gold['ID'].isin(df_walk['ID'])]
gold_cmplx_sizes = df_gold.groupby('cmplx').size()

In [6]:
df_wt_gold = df_walk[df_walk.ID.isin(df_gold.ID.tolist())]
df_wt_gold = df_wt_gold.merge(df_gold, how='left', left_on=['ID'], right_on=['ID']).sort_values('cmplx')
df_wt_gold

,ID,cut_154,cut_298,cut_442,cut_817,cut_1193,cut_1568,cut_1944,cut_2320,cut_2695,cut_3071,cmplx
3120,KOG3599,4,6,11,20,292,772,1119,1432,1740,2064,1
822,KOG4203,1,1,12,77,118,353,445,705,1190,1409,1
757,KOG0676,1,1,12,77,118,164,200,249,298,360,2
611,KOG0101,1,1,12,60,90,124,275,641,765,910,2
570,KOG0730,1,1,12,56,208,299,440,761,922,1090,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1588,KOG4046,1,2,2,4,5,8,9,12,202,245,1495
1827,KOG0018,1,2,7,11,45,57,70,93,112,134,1496
3610,KOG1133,4,6,11,208,368,554,752,978,1192,1411,1496
349,KOG0785,1,1,6,115,190,261,322,397,473,563,1497


In [20]:
df_clst_gold = df_walk.merge(df_gold, how='inner', left_on=['ID'], right_on=['ID']).sort_values('cmplx')
df_clst_gold

,ID,cut_154,cut_298,cut_442,cut_817,cut_1193,cut_1568,cut_1944,cut_2320,cut_2695,cut_3071,cmplx
3120,KOG3599,4,6,11,20,292,772,1119,1432,1740,2064,1
822,KOG4203,1,1,12,77,118,353,445,705,1190,1409,1
757,KOG0676,1,1,12,77,118,164,200,249,298,360,2
611,KOG0101,1,1,12,60,90,124,275,641,765,910,2
570,KOG0730,1,1,12,56,208,299,440,761,922,1090,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1588,KOG4046,1,2,2,4,5,8,9,12,202,245,1495
1827,KOG0018,1,2,7,11,45,57,70,93,112,134,1496
3610,KOG1133,4,6,11,208,368,554,752,978,1192,1411,1496
349,KOG0785,1,1,6,115,190,261,322,397,473,563,1497


In [16]:
df_list = []
for c in cuts:
    cut_df = df_wt_gold[['ID', c, 'cmplx']]
    cmplx_counts = cut_df.groupby([c, 'cmplx']).size().reset_index(name='counts')
    prop_list = []
    cmplx_list = []
    for i in range(len(cmplx_counts)):
        cmplx_num = cmplx_counts['cmplx'][i]
        obs = cmplx_counts['counts'][i]
        actual = gold_cmplx_sizes[cmplx_num]
        # calculate proportion of obs gold std complex in same cluster
        prop = round(obs/actual, 2)
        # record results
        cmplx_list.append(cmplx_num)
        prop_list.append(prop)
    cut_res = pd.DataFrame()
    cut_res['cmplx'] = cmplx_list
    cut_res[c] = prop_list
    max_obs = cut_res.loc[cut_res.groupby(['cmplx'])[c].idxmax()]
    df_list.append(max_obs)

In [17]:
for df in df_list:
    df.set_index(['cmplx'], inplace=True)
final_df = reduce(lambda x, y: x.join(y, how='outer'), df_list)
final_df = final_df.fillna(0).reset_index()

In [18]:
final_df

,cmplx,cut_154,cut_298,cut_442,cut_817,cut_1193,cut_1568,cut_1944,cut_2320,cut_2695,cut_3071
0,1,0.5,0.5,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
1,2,1.0,1.0,1.00,0.50,0.25,0.25,0.25,0.25,0.25,0.25
2,3,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
3,4,0.5,0.5,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
4,5,1.0,1.0,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
...,...,...,...,...,...,...,...,...,...,...,...
1271,1493,1.0,0.5,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
1272,1494,1.0,1.0,1.00,0.50,0.50,0.50,0.50,0.50,0.50,0.50
1273,1495,1.0,1.0,0.83,0.67,0.67,0.50,0.33,0.17,0.17,0.17
1274,1496,0.5,0.5,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50


In [195]:
obs_sizes = df_wt_gold.groupby(['cut_442', 'cmplx']).size().reset_index(name='obs_size')
res_list = []
for i in range(len(obs_sizes)):
    cmplx_num = obs_sizes['cmplx'][i]
    obs = obs_sizes['obs_size'][i]
    actual = gold_cmplx_sizes[cmplx_num]
    prop = obs/actual
    res_list.append(prop)
obs_sizes['proportion'] = res_list
obs_sizes

,cut_442,cmplx,obs_size,proportion
0,0,51,1,0.333333
1,0,107,1,0.333333
2,0,436,1,0.500000
3,0,455,18,1.000000
4,0,522,22,1.000000
...,...,...,...,...
2095,389,620,1,0.500000
2096,410,946,1,0.250000
2097,431,713,1,0.034483
2098,435,149,1,0.333333


In [38]:
test_dict = dict()
scores = [0.1, -0.78, 0.66, -0.33, 0.99]
for i, s in enumerate(scores):
    test_dict[i+1] = s
test_dict

{1: 0.1, 2: -0.78, 3: 0.66, 4: -0.33, 5: 0.99}

In [46]:
df = pd.DataFrame.from_dict(test_dict, orient='index')
df = df.reset_index().rename(columns={'index':'fold', 0:'average_precision'})
df

,fold,average_precision
0,1,0.10
1,2,-0.78
2,3,0.66
3,4,-0.33
4,5,0.99


In [43]:
import pickle
import pandas as pd
from itertools import combinations
import time
from datetime import datetime as dt

pos_file = '../ppi_ml/data/featmats/positive_ppi_dict.pkl'
neg_file = '../ppi_ml/data/featmats/negative_ppi_dict.pkl'
walk_file = '../ppi_ml/results/walktrap/step_sweep/SGDClassifier_25feats_fdr10_3steps.csv'

with open(pos_file, 'rb') as f:
    pos_dict = pickle.load(f)
with open(neg_file, 'rb') as f:
    neg_dict = pickle.load(f)
    
clst_res = pd.read_csv(walk_file)
cuts = [c for c in res.columns.values.tolist() if 'cut' in c]

In [11]:
df_walk = clst_res[['ID'] + cuts]
df_walk

,ID,cut_60,cut_110,cut_161,cut_676,cut_1192,cut_1707,cut_2223,cut_2738,cut_3254,cut_3769
0,KOG2241,0,0,0,0,0,0,0,0,0,0
1,KOG1885,0,0,0,0,0,0,0,0,1,1
2,KOG0437,0,0,0,0,0,0,0,0,936,1160
3,KOG4426,0,0,0,0,0,0,0,0,936,1238
4,KOG0434,0,0,0,0,0,0,0,888,1133,1418
...,...,...,...,...,...,...,...,...,...,...,...
4280,KOG4499,55,105,156,671,1187,1702,2218,2733,3249,3764
4281,KOG0956,56,106,157,672,1188,1703,2219,2734,3250,3765
4282,ENOG502QQFM,57,107,158,673,1189,1704,2220,2735,3251,3766
4283,ENOG502QRAK,58,108,159,674,1190,1705,2221,2736,3252,3767


In [65]:
neg_ppis = list(neg_dict.keys())
len(neg_ppis)

19887

In [66]:
pos_ppis = [item for sublist in list(pos_dict.values()) for item in sublist]
len(pos_ppis)

18612

In [68]:
t0 = time.time()
ogs = df_walk['ID'].tolist()
all_pairs = [frozenset({i, j}) for i,j in list(combinations(ogs, 2))]
print(f'Total number of pairs: {len(all_pairs)}')

print(f'Getting all potential positive PPIs in results ...')
all_obs_pos = set(all_pairs).intersection(set(pos_ppis))
total_obs_pos = len(all_obs_pos)
print('Total possible positive PPIs:', total_obs_pos)

print(f'Getting all potential negative PPIs in results ...')
all_obs_neg = set(all_pairs).intersection(set(neg_ppis))
total_obs_neg = len(all_obs_neg)
print('Total possible negative PPIs:', total_obs_neg)

rt = round((time.time()-t0)/60, 2)
print('---------------------------------------')
print('Total time to calculate:', rt, 'minutes')
print('---------------------------------------')

Total number of pairs: 9178470
Getting all potential positive PPIs in results ...
Total possible positive PPIs: 8005
Getting all potential negative PPIs in results ...
Total possible negative PPIs: 19632
---------------------------------------
Total time to calculate: 0.38 minutes
---------------------------------------


In [81]:
def get_ppis(df, col, all_pos, all_neg):
    df = df[['ID', col]]
    grouped = df.groupby(col)
    pos_set = set()
    neg_set = set()
    for cluster, data in grouped:
        prot_lst = [p for p in data['ID']]
        ppis = [frozenset({i, j}) for i,j in list(combinations(prot_lst, 2))]
        pos = set(ppis).intersection(set(all_pos))
        pos_set.update(set(pos))
        neg = set(ppis).intersection(set(all_neg))
        neg_set.update(set(neg))
    return(pos_set, neg_set)

def calc_cut_pr(pos, neg, total_pos):
    tp = len(pos)
    fp = len(neg)
    fn = total_pos - tp
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    return(precision, recall)

In [85]:
t0 = time.time()
print(f'Calculating precision, recall for {len(cuts)} cuts ...')
res_dict = dict()
for c in cuts:
    n_clst = c.replace('cut_','') 
    pos, neg = get_ppis(df_walk, c, all_obs_pos, all_obs_neg)
    precision, recall = calc_cut_pr(pos, neg, total_obs_pos)
    res_dict[n_clst] = [float(precision), float(recall)]
    print(f'{n_clst} clusters: P={precision}, R={recall}')
    
res_df = pd.DataFrame.from_dict(res_dict, orient='index', columns=['precision','recall'])
res_df = res_df.reset_index().rename(columns={'index': 'cut'})
print(res_df)

rt = round((time.time()-t0)/60, 2)
print('---------------------------------------')
print(f'Total time to calculate for {len(cuts)} cuts: {rt} minutes')
print('---------------------------------------')

Calculating precision, recall for 10 cuts ...
60 clusters: P=0.28988975921090804, R=0.9986258588382261
110 clusters: P=0.2895646164478231, R=0.9945034353529044
161 clusters: P=0.4109061864652919, R=0.7069331667707682
676 clusters: P=0.7803138373751783, R=0.27332916926920675
1192 clusters: P=0.8557806912991657, R=0.17938788257339164
1707 clusters: P=0.9066666666666666, R=0.1444097439100562
2223 clusters: P=0.926954732510288, R=0.11255465334166147
2738 clusters: P=0.9418803418803419, R=0.06883198001249219
3254 clusters: P=0.9385113268608414, R=0.03622735790131168
3769 clusters: P=0.9380530973451328, R=0.013241723922548408
   cuts  precision    recall
0    60   0.289890  0.998626
1   110   0.289565  0.994503
2   161   0.410906  0.706933
3   676   0.780314  0.273329
4  1192   0.855781  0.179388
5  1707   0.906667  0.144410
6  2223   0.926955  0.112555
7  2738   0.941880  0.068832
8  3254   0.938511  0.036227
9  3769   0.938053  0.013242
---------------------------------------
Total time to

In [7]:
import pickle
import pandas as pd
import numpy as np

In [5]:
fm_pkl = '../ppi_ml/data/featmats/featmat.pkl'
with open(fm_pkl, 'rb') as f:
    fm = pickle.load(f)
fm.head()

,ID,arath.iex_1.150p.braycurtis.feat,arath.iex_1.150p.euclidean.feat,arath.iex_1.150p.pearsonR.feat,arath.iex_1.150p.spearmanR.feat,arath.iex_2.150p.braycurtis.feat,arath.iex_2.150p.euclidean.feat,arath.iex_2.150p.pearsonR.feat,arath.iex_2.150p.spearmanR.feat,arath.iex_3.150p.braycurtis.feat,...,neg_ln_pval_treiber_hygeo_gt2,pair_count_treiber_hygeo_gt2,plants_concat.raw.150p.spearmanR.feat,plants_concat.raw.150p.pearsonR.feat,plants_concat.raw.150p.braycurtis.feat,plants_concat.raw.150p.euclidean.feat,tsar_concat.raw.150p.spearmanR.feat,tsar_concat.raw.150p.braycurtis.feat,tsar_concat.raw.150p.pearsonR.feat,tsar_concat.raw.150p.euclidean.feat
0,ENOG502QPIC ENOG502QR7H,0.9804,5.1570,-0.0755,-0.0663,0.9713,9.5771,-0.0672,-0.0157,0.9975,...,0.0,0.0,-0.0338,-0.0287,0.0350,0.3934,0.2800,0.2383,0.2750,0.7874
1,KOG0253 ENOG502QPIC,0.9974,5.4920,-0.0463,-0.0597,0.0000,0.0000,0.0000,0.0000,0.8553,...,0.0,0.0,0.0643,0.0442,0.1303,0.3670,-0.0272,0.0095,-0.0238,0.4956
2,ENOG502QPIC KOG0734,0.8650,5.9164,0.1033,0.0950,0.9990,11.5670,-0.1392,-0.1714,0.6362,...,0.0,0.0,0.0206,-0.0230,0.0472,0.3953,0.0134,0.0287,-0.0112,0.4943
3,KOG0739 ENOG502QPIC,0.9307,5.5960,0.0759,0.0713,0.8998,9.2430,0.0841,0.1159,0.8795,...,0.0,0.0,0.0389,0.0016,0.0726,0.3898,0.0524,0.0157,-0.0183,0.6353
4,KOG0935 ENOG502QPIC,0.9969,4.6944,-0.0366,-0.0419,0.0000,0.0000,0.0000,0.0000,0.9974,...,0.0,0.0,0.0387,0.0212,0.1362,0.3848,0.4342,0.2687,0.1434,0.6477


In [24]:
col_patterns = ['animals', 'plants', 'tsar', 'excavate', 'leca']
cols = []
for p in col_patterns:
    res = fm.filter(like=p)
    cols.append(res)
df = pd.concat(cols, axis=1, ignore_index=False)
df.columns = df.columns.str.replace("_concat.raw.150p", "")
df.columns = df.columns.str.replace("_euks_elut.filtdollo.filt150p.raw", "")
df.columns = df.columns.str.replace(".feat", "")
df.columns = df.columns.str.replace("excavate", "exca")
df.columns = df.columns.str.replace("animals", "amor")
df.columns = df.columns.str.replace("plants", "viri")
df

/tmp/ipykernel_2619727/477528682.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace("_concat.raw.150p", "")
/tmp/ipykernel_2619727/477528682.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace("_euks_elut.filtdollo.filt150p.raw", "")
/tmp/ipykernel_2619727/477528682.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(".feat", "")


,amor.pearsonR,amor.spearmanR,amor.euclidean,amor.braycurtis,viri.spearmanR,viri.pearsonR,viri.braycurtis,viri.euclidean,tsar.spearmanR,tsar.braycurtis,tsar.pearsonR,tsar.euclidean,exca.euclidean,exca.pearsonR,exca.braycurtis,exca.spearmanR,leca.pearsonR,leca.euclidean,leca.spearmanR,leca.braycurtis
0,0.0000,0.0000,0.0000,0.0000,-0.0338,-0.0287,0.0350,0.3934,0.2800,0.2383,0.2750,0.7874,0.0000,0.0000,0.0000,0.0000,0.0125,0.5521,0.0916,0.0453
1,0.0000,0.0000,0.0000,0.0000,0.0643,0.0442,0.1303,0.3670,-0.0272,0.0095,-0.0238,0.4956,0.0000,0.0000,0.0000,0.0000,0.0224,0.3975,0.0479,0.0833
2,0.0000,0.0000,0.0000,0.0000,0.0206,-0.0230,0.0472,0.3953,0.0134,0.0287,-0.0112,0.4943,0.0000,0.0000,0.0000,0.0000,0.0055,0.5185,0.0561,0.0393
3,0.0000,0.0000,0.0000,0.0000,0.0389,0.0016,0.0726,0.3898,0.0524,0.0157,-0.0183,0.6353,0.0000,0.0000,0.0000,0.0000,-0.0048,0.4611,0.0095,0.0323
4,0.0000,0.0000,0.0000,0.0000,0.0387,0.0212,0.1362,0.3848,0.4342,0.2687,0.1434,0.6477,0.0000,0.0000,0.0000,0.0000,0.0314,0.5324,0.1076,0.0725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491713,0.0021,-0.0040,0.6692,0.0240,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8082,0.0739,0.4030,0.3396,0.0579,0.6626,0.2029,0.1345
4491714,0.0003,0.0275,0.8331,0.0917,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7964,0.1882,0.4652,0.2871,0.1464,0.7825,0.3331,0.3038
4491715,0.0002,0.0133,0.6887,0.0246,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7138,0.1556,0.4831,0.2759,0.1444,0.7022,0.3287,0.2833
4491716,0.0289,0.0657,0.8109,0.0660,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5940,-0.0049,0.2952,-0.0169,0.0467,0.6939,0.3643,0.1816


In [25]:
corr = df.corr()
corr = round(corr, 2)

In [26]:
corr

,amor.pearsonR,amor.spearmanR,amor.euclidean,amor.braycurtis,viri.spearmanR,viri.pearsonR,viri.braycurtis,viri.euclidean,tsar.spearmanR,tsar.braycurtis,tsar.pearsonR,tsar.euclidean,exca.euclidean,exca.pearsonR,exca.braycurtis,exca.spearmanR,leca.pearsonR,leca.euclidean,leca.spearmanR,leca.braycurtis
amor.pearsonR,1.00,0.59,0.06,0.86,0.20,0.21,0.19,0.02,0.10,0.12,0.11,0.01,0.06,0.13,0.11,0.13,0.52,-0.05,0.27,0.42
amor.spearmanR,0.59,1.00,0.03,0.80,0.33,0.21,0.29,-0.05,0.17,0.19,0.11,0.03,0.12,0.10,0.17,0.16,0.33,-0.11,0.45,0.45
amor.euclidean,0.06,0.03,1.00,0.15,-0.05,-0.02,-0.06,0.07,-0.04,-0.05,-0.02,0.01,0.03,0.01,-0.01,0.01,-0.01,0.39,0.00,-0.00
amor.braycurtis,0.86,0.80,0.15,1.00,0.28,0.23,0.26,-0.01,0.14,0.17,0.11,0.02,0.10,0.13,0.17,0.17,0.47,-0.07,0.36,0.51
viri.spearmanR,0.20,0.33,-0.05,0.28,1.00,0.66,0.84,0.12,0.16,0.17,0.10,0.04,0.09,0.10,0.15,0.16,0.32,-0.12,0.49,0.45
viri.pearsonR,0.21,0.21,-0.02,0.23,0.66,1.00,0.79,0.14,0.07,0.10,0.09,-0.00,0.03,0.11,0.08,0.12,0.42,-0.04,0.28,0.38
viri.braycurtis,0.19,0.29,-0.06,0.26,0.84,0.79,1.00,0.17,0.14,0.16,0.09,0.02,0.08,0.09,0.14,0.14,0.36,-0.16,0.43,0.47
viri.euclidean,0.02,-0.05,0.07,-0.01,0.12,0.14,0.17,1.00,-0.11,-0.12,-0.09,-0.05,-0.01,-0.00,-0.02,-0.01,-0.00,0.22,-0.03,-0.02
tsar.spearmanR,0.10,0.17,-0.04,0.14,0.16,0.07,0.14,-0.11,1.00,0.84,0.75,0.38,0.12,0.07,0.12,0.10,0.27,-0.05,0.36,0.33
tsar.braycurtis,0.12,0.19,-0.05,0.17,0.17,0.10,0.16,-0.12,0.84,1.00,0.90,0.46,0.18,0.12,0.21,0.17,0.34,-0.10,0.37,0.40


In [27]:
corr_tbl = corr.style.background_gradient(cmap='coolwarm').format(precision=2)

In [28]:
corr_tbl

,amor.pearsonR,amor.spearmanR,amor.euclidean,amor.braycurtis,viri.spearmanR,viri.pearsonR,viri.braycurtis,viri.euclidean,tsar.spearmanR,tsar.braycurtis,tsar.pearsonR,tsar.euclidean,exca.euclidean,exca.pearsonR,exca.braycurtis,exca.spearmanR,leca.pearsonR,leca.euclidean,leca.spearmanR,leca.braycurtis
amor.pearsonR,1.00,0.59,0.06,0.86,0.20,0.21,0.19,0.02,0.10,0.12,0.11,0.01,0.06,0.13,0.11,0.13,0.52,-0.05,0.27,0.42
amor.spearmanR,0.59,1.00,0.03,0.80,0.33,0.21,0.29,-0.05,0.17,0.19,0.11,0.03,0.12,0.10,0.17,0.16,0.33,-0.11,0.45,0.45
amor.euclidean,0.06,0.03,1.00,0.15,-0.05,-0.02,-0.06,0.07,-0.04,-0.05,-0.02,0.01,0.03,0.01,-0.01,0.01,-0.01,0.39,0.00,-0.00
amor.braycurtis,0.86,0.80,0.15,1.00,0.28,0.23,0.26,-0.01,0.14,0.17,0.11,0.02,0.10,0.13,0.17,0.17,0.47,-0.07,0.36,0.51
viri.spearmanR,0.20,0.33,-0.05,0.28,1.00,0.66,0.84,0.12,0.16,0.17,0.10,0.04,0.09,0.10,0.15,0.16,0.32,-0.12,0.49,0.45
viri.pearsonR,0.21,0.21,-0.02,0.23,0.66,1.00,0.79,0.14,0.07,0.10,0.09,-0.00,0.03,0.11,0.08,0.12,0.42,-0.04,0.28,0.38
viri.braycurtis,0.19,0.29,-0.06,0.26,0.84,0.79,1.00,0.17,0.14,0.16,0.09,0.02,0.08,0.09,0.14,0.14,0.36,-0.16,0.43,0.47
viri.euclidean,0.02,-0.05,0.07,-0.01,0.12,0.14,0.17,1.00,-0.11,-0.12,-0.09,-0.05,-0.01,-0.00,-0.02,-0.01,-0.00,0.22,-0.03,-0.02
tsar.spearmanR,0.10,0.17,-0.04,0.14,0.16,0.07,0.14,-0.11,1.00,0.84,0.75,0.38,0.12,0.07,0.12,0.10,0.27,-0.05,0.36,0.33
tsar.braycurtis,0.12,0.19,-0.05,0.17,0.17,0.10,0.16,-0.12,0.84,1.00,0.90,0.46,0.18,0.12,0.21,0.17,0.34,-0.10,0.37,0.40


In [34]:
from Bio import SeqIO

# specify fasta file
fa_file = '../ppi_ml/data/proteomes/human.fasta'

# if you only want to keep specific IDs, put them in a list
subset_ids = ['sp|A0A024RBG1|NUD4B_HUMAN','sp|A0A075B6I0|LV861_HUMAN','sp|A0A075B6I3|LVK55_HUMAN']

# specify outfile name to write subset to
out_file = '../../misc/test_subset.fasta'

# read in records & extract the ones that match the subset
fasta = SeqIO.parse(fa_file, "fasta")
records_subset = [rec for rec in fasta if rec.id in subset_ids]
SeqIO.write(records_subset, out_file, "fasta")

In [40]:
[print(i.id+'\n'+i.seq) for i in records_subset]

sp|A0A024RBG1|NUD4B_HUMAN
MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGIFEQNQDRKHRTYVYVLTVTEILEDWEDSVNIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGCSPANGNSTVPSLPDNNALFVTAAQTSGLPSSVR
sp|A0A075B6I0|LV861_HUMAN
MSVPTMAWMMLLLGLLAYGSGVDSQTVVTQEPSFSVSPGGTVTLTCGLSSGSVSTSYYPSWYQQTPGQAPRTLIYSTNTRSSGVPDRFSGSILGNKAALTITGAQADDESDYYCVLYMGSGI
sp|A0A075B6I3|LVK55_HUMAN
MALTPLLLLLLSHCTGSLSRPVLTQPPSLSASPGATARLPCTLSSDLSVGGKNMFWYQQKLGSSPRLFLYHYSDSDKQLGPGVPSRVSGSKETSSNTAFLLISGLQPEDEADYYCQVYESSAN


[None, None, None]